In [8]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score
from fairlearn.metrics import MetricFrame
from sklearn import metrics as skm
import seaborn as sns
import os.path as path
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point, Polygon
import os, csv, shapely, shapely.wkt, descartes, geopy, pickle, cloudpickle
from os import listdir
from os.path import isfile, join
pd.set_option('display.max_rows', 10)
sns.set()
sns.color_palette("ch:s=.25,rot=-.25")
np.set_printoptions(suppress=True)
# pd.options.display.float_format = '{:.2f}'.format

In [34]:
NUM_STATIONS_INC = 2

In [35]:
# NEED MODIFY
EPOCH = 120
LAMBDA = 0.2
FOLDER = 'fairst_all_models/e_120_lambda_0_2'
PRED_FILE_NAME = "inference_stations_2_fused_model_pred_0.2_1641298395.7021697.csv"
# # NEED MODIFY
# EPOCH = 120
# LAMBDA = 0.2
# FOLDER = 'fairst_all_models/forecast_e_120_lambda_0_2'
# PRED_FILE_NAME = "fused_model_pred_0.2_1640929879.8990524.csv"

In [36]:
CITY = "P_"
save_dir = CITY + 'res/'
save_dir_preprocess = os.path.join(save_dir, CITY + 'preprocess_fig/')
save_dir_models = os.path.join(save_dir, CITY + 'model_res/') 
save_dir_forecast = os.path.join(save_dir, CITY + 'forecast/')
save_dir_fairst = os.path.join(save_dir, CITY + 'fairst/')

if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
if not os.path.exists(save_dir_preprocess):
    os.makedirs(save_dir_preprocess)

if not os.path.exists(save_dir_models):
    os.makedirs(save_dir_models)
    
if not os.path.exists(save_dir_forecast):
    os.makedirs(save_dir_forecast)
    
if not os.path.exists(save_dir_fairst):
    os.makedirs(save_dir_fairst)

In [37]:
pred_df = pd.read_csv(os.path.join(save_dir_fairst, FOLDER, PRED_FILE_NAME), index_col = 0)

pos_info = pd.read_csv(os.path.join(save_dir_fairst, "df_demo.csv"), index_col = 0)
income_info = pd.read_csv(os.path.join(save_dir_fairst, "df_income.csv"), index_col = 0)
income_info = income_info.groupby(by = "cell_num").mean().reset_index()
income_info.income_group = income_info.income_group.astype(int)
demo_raw = pd.read_csv(os.path.join(save_dir_fairst, "demo_raw.csv"))
normalized_pop_df = pd.read_csv(os.path.join(save_dir_fairst, "normalized_pop_df.csv"), index_col = 0)
stations_info = pd.read_csv(os.path.join(save_dir_fairst, "pgh_grid_census.csv"), index_col = 0)[["cell_num", "stations"]]
pred_df.index = pd.to_datetime(pred_df.index)


In [38]:
# def df_transform(df):
#     pos_l = df.columns
#     df_trans = pd.DataFrame(columns = ["pos", "outflow"])

#     for i in np.arange(len(pos_l)):
#         pos = pos_l[i]
#         temp = df[pos].to_frame()
#         temp["pos"] = [temp.columns.values[0]]*len(temp)
#         temp.rename(columns = {pos: "outflow"}, inplace = True)
#         df_trans = df_trans.append(temp)
        
#     return df_trans

In [39]:
# # original
# pred_df_month = pred_df.groupby(pred_df.index.month).sum()
# df_pred_trans = df_transform(pred_df_month)
# df_pred_trans_m = df_pred_trans.groupby("pos").sum().reset_index()
# pred_final = df_pred_trans_m.merge(pos_info)
# pred_final_m = pred_final.merge(income_info, on = 'cell_num')
# for row, col in pred_final_m.iterrows():
#     pred_final_m.at[row, 'geometry'] = shapely.wkt.loads(pred_final_m.loc[row]['geometry'])
# pred_final_m = gpd.GeoDataFrame(pred_final_m, crs='EPSG:2956', geometry='geometry')

In [40]:
# # add 1
# pred_df_month_1 = pred_df_1.groupby(pred_df_1.index.month).sum()
# df_pred_trans_1 = df_transform(pred_df_month_1)
# df_pred_trans_m_1 = df_pred_trans_1.groupby("pos").sum().reset_index()
# pred_final_1 = df_pred_trans_m_1.merge(pos_info)
# pred_final_m_1 = pred_final_1.merge(income_info, on = 'cell_num')

# for row, col in pred_final_m_1.iterrows():
#     pred_final_m_1.at[row, 'geometry'] = shapely.wkt.loads(pred_final_m_1.loc[row]['geometry'])
    
# pred_final_m_1 = gpd.GeoDataFrame(pred_final_m_1, crs='EPSG:2956', geometry='geometry')

In [41]:
# pred_final_m.plot(column="outflow", cmap='Blues',edgecolor = "grey", figsize=(8, 10),  
#                     legend = True, legend_kwds={'label': "outflow",'orientation': "horizontal"})
# plt.title("fairST\n(before add 1 station)", fontsize=20)

In [42]:
# pred_final_m_1.plot(column="outflow", cmap='Blues',edgecolor = "grey", figsize=(8, 10),  
#                     legend = True, legend_kwds={'label': "outflow",'orientation': "horizontal"})
# plt.title("Unmitigated random forest forecast\n(before add 1 station)", fontsize=20)

### calc

In [43]:
# def forecast_res_calc(final_res):
#     diff_col = final_res.columns[-3]
#     # print(diff_col)
    
#     high_inc = final_res[final_res["income_group"] == 1]
#     high_inc_sum_diff = (high_inc["outflow_diff"] / high_inc["stations"]).sum()
#     high_inc_sum_diff_avg = high_inc_sum_diff/len(high_inc)
    
#     low_inc = final_res[final_res["income_group"] == 0]
#     low_inc_sum_diff = (low_inc["outflow_diff"] / low_inc["stations"]).sum()
#     low_inc_sum_diff_avg = low_inc_sum_diff/len(low_inc)
    
# #     print(high_inc_sum_diff_avg)
# #     print(low_inc_sum_diff_avg)
#     diff = high_inc_sum_diff_avg - low_inc_sum_diff_avg
#     ratio = low_inc_sum_diff_avg / high_inc_sum_diff_avg
    
#     return high_inc_sum_diff_avg, low_inc_sum_diff_avg, diff, ratio

In [44]:
# pred_final_m = pred_final_m.sort_values('cell_num')
# pred_final_m_1 = pred_final_m_1.sort_values('cell_num')

# final_res = pd.DataFrame(columns = ['pos', 'outflow_before', 'outflow_after', "outflow_diff", "cell_num", "income_group", "geometry"])

# final_res['pos'] = pred_final_m.pos
# final_res['outflow_before'] = pred_final_m.outflow
# final_res['outflow_after'] = pred_final_m_1.outflow
# final_res['outflow_diff'] = pred_final_m_1.outflow - pred_final_m.outflow
# final_res['cell_num'] = pred_final_m.cell_num
# final_res['income_group'] = pred_final_m.income_group
# final_res['geometry'] = pred_final_m.geometry
# final_res = final_res.merge(stations_info, on = 'cell_num')
# final_res.stations += NUM_STATIONS_ADDED

In [45]:
# # for row, col in final_res.iterrows():
# #     final_res.at[row, 'geometry'] = shapely.wkt.loads(final_res.loc[row]['geometry'])
    
# final_res = gpd.GeoDataFrame(final_res, crs='EPSG:2956', geometry='geometry')

In [46]:
# final_res.plot(column="outflow_diff", cmap='Blues',edgecolor = "grey", figsize=(8, 10),  
#                     legend = True, legend_kwds={'label': "outflow",'orientation': "horizontal"})
# plt.title("Unmitigated random forest forecast\n(before add 1 station)", fontsize=20)

___

In [47]:
def df_transform(df):
    pos_l = df.columns
    df_trans = pd.DataFrame(columns = ["pos", "outflow"])

    for i in np.arange(len(pos_l)):
        pos = pos_l[i]
        temp = df[pos].to_frame()
        temp["pos"] = [temp.columns.values[0]]*len(temp)
        temp.rename(columns = {pos: "outflow"}, inplace = True)
        df_trans = df_trans.append(temp)
        
    return df_trans

In [48]:
pred_df_month = pred_df.groupby(pred_df.index.month).sum()

In [50]:
df_transform_m = df_transform(pred_df_month).groupby('pos').sum()
df_transform_m = df_transform_m.reset_index()

In [51]:
# df_pred_trans = df_transform(pred_df_month)
pred_final = df_transform_m.merge(pos_info)

pred_final_m = pred_final.merge(income_info, on = 'cell_num')

In [52]:
pred_final_m

,pos,outflow,cell_num,geometry,income_group
0,10_14,0.000000,334,POLYGON ((-80.05401960776868 40.42588145854761...,1
1,10_15,0.002862,335,POLYGON ((-80.05401960776868 40.43047224708476...,1
2,10_16,0.000000,336,POLYGON ((-80.05401960776868 40.43506303562191...,1
3,10_17,0.000000,337,POLYGON ((-80.05401960776868 40.43965382415906...,1
4,10_18,0.000000,338,POLYGON ((-80.05401960776868 40.44424461269621...,0
...,...,...,...,...,...
723,9_17,0.000000,305,POLYGON ((-80.05861039630584 40.43965382415906...,1
724,9_18,0.000000,306,POLYGON ((-80.05861039630584 40.44424461269621...,0
725,9_19,0.000000,307,POLYGON ((-80.05861039630584 40.44883540123336...,0
726,9_20,0.150453,308,POLYGON ((-80.05861039630584 40.45342618977051...,0


In [53]:
pred_final_m.to_csv(os.path.join(save_dir_fairst, "fairst_forecast_inc_{}.csv".format(NUM_STATIONS_INC)))